In [20]:
from praw import Reddit
from praw.models import Subreddit, Submission
import os
from dotenv import load_dotenv
from datetime import datetime, date, timedelta
from dotenv import dotenv_values
import datetime as dt
import pandas as pd
from datetime import datetime, date, timedelta

In [29]:
import praw
from datetime import datetime
from dotenv import dotenv_values
import pandas as pd

# Load environment variables
config = dotenv_values(".env")

# Initialize Reddit API
reddit = praw.Reddit(
    client_id=config["CLIENT_ID"],
    client_secret=config["CLIENT_SECRET"],
    redirect_uri=config["REDIRECT_URL"],
    user_agent=config["USER_AGENT"]
)

def fetch_posts_about_keyword(subreddit_name, keywords, limit=500000):
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    
    # Define the cutoff date (before November 2024)
    cutoff_date = datetime(2024, 11, 30)  # November 1, 2024
    
    # Fetch posts from subreddit
    for post in subreddit.search(' '.join(keywords), time_filter='all', limit=limit):
        post_date = datetime.utcfromtimestamp(post.created)
        
        # Check if the post is before the cutoff date
        if post_date < cutoff_date:
            matching_keywords = [keyword for keyword in keywords if keyword.lower() in post.title.lower()]

            for keyword in matching_keywords:
                post.comments.replace_more(limit=0)
                comments = post.comments.list()

                for comment in comments:
                    data.append({
                        'post_title': post.title,
                        'post_score': post.score,
                        'post_id': post.id,
                        'post_num_comments': post.num_comments,
                        'post_created': post_date,
                        'post_body': post.selftext,
                        'comment_id': comment.id,
                        'comment_body': comment.body,
                        'comment_score': comment.score,
                        'matched_keywords': ", ".join(matching_keywords)
                    })
                break  # Only process the post once per matching keyword

    return data

subreddit_name = "stocks"
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]

# Fetch posts before November 2024
data = fetch_posts_about_keyword(subreddit_name, keywords, limit=1000)

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv("posts_before_november_2024.csv", index=False)


In [42]:
# Load environment variables
config = dotenv_values(".env")

# Initialize Reddit API
reddit = praw.Reddit(
    client_id=config["CLIENT_ID"],
    client_secret=config["CLIENT_SECRET"],
    user_agent=config["USER_AGENT"]
)

def fetch_posts_about_keywords(subreddit_name, keywords, limit=500000):
    subreddit = reddit.subreddit(subreddit_name)
    data = []
    
    # Define the date range for March and April (2024)
    today = datetime(2024, 9, 30, 23, 59, 59)  # End of April
    start_date = datetime(2024, 7, 1)  # Start of March
    
    # Iterate through each keyword
    for keyword in keywords:
        print(f"Fetching posts for keyword: {keyword}")
        for post in subreddit.search(keyword, time_filter='all', limit=limit):
            post_date = datetime.utcfromtimestamp(post.created)
            
            # Check if the post is within the desired time range
            if start_date <= post_date <= today:
                post.comments.replace_more(limit=0)
                comments = post.comments.list()

                for comment in comments:
                    data.append({
                        'post_title': post.title,
                        'post_score': post.score,
                        'post_id': post.id,
                        'post_num_comments': post.num_comments,
                        'post_created': post_date,
                        'post_body': post.selftext,
                        'comment_id': comment.id,
                        'comment_body': comment.body,
                        'comment_score': comment.score,
                        'matched_keyword': keyword
                    })
    
    return data

subreddit_name = "stocks"
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]

# Fetch posts from March and April 2024
data = fetch_posts_about_keywords(subreddit_name, keywords, limit=500000)

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv("before_september_reddit_comments.csv", index=False)


Fetching posts for keyword: Uber
Fetching posts for keyword: youtube
Fetching posts for keyword: meta
Fetching posts for keyword: apple
Fetching posts for keyword: nvidia
Fetching posts for keyword: microsoft
Fetching posts for keyword: amazon
Fetching posts for keyword: Saudi Arabian Oil Co
Fetching posts for keyword: intel
Fetching posts for keyword: tesla
Fetching posts for keyword: MARA Holdings


In [59]:
full_df = pd.read_csv("full_dataset.csv")
full_df['company'] = full_df['matched_keyword'].fillna(full_df['matched_keywords'])
full_df.drop(columns=['matched_keyword', 'matched_keywords'], inplace=True)

C:\Users\Chaimaa\AppData\Local\Temp\ipykernel_26072\1616194821.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv("full_dataset.csv")


In [64]:
full_df

,post_title,post_score,post_id,post_num_comments,post_created,post_body,comment_id,comment_body,comment_score,company
0,Tesla stands to earn billions by opening U.S. ...,548,1b3kl34,175,2024-03-01 03:45:26,"As of Thursday, owners of Ford electric vehicl...",kswcta9,It’s the country i live in and it’s natural fo...,1,tesla
1,Tesla stands to earn billions by opening U.S. ...,548,1b3kl34,175,2024-03-01 03:45:26,"As of Thursday, owners of Ford electric vehicl...",ksufbu2,https://en.wikipedia.org/wiki/North_American_C...,7,tesla
2,Tesla stands to earn billions by opening U.S. ...,548,1b3kl34,175,2024-03-01 03:45:26,"As of Thursday, owners of Ford electric vehicl...",ksv9blq,Very good point.,1,tesla
3,Tesla stands to earn billions by opening U.S. ...,548,1b3kl34,175,2024-03-01 03:45:26,"As of Thursday, owners of Ford electric vehicl...",ksts85g,If they can stay alive long enough. Most of th...,6,tesla
4,Tesla stands to earn billions by opening U.S. ...,548,1b3kl34,175,2024-03-01 03:45:26,"As of Thursday, owners of Ford electric vehicl...",ksuj8j5,Google software 🤣\nIt won't fix the laggy sinc...,3,tesla
...,...,...,...,...,...,...,...,...,...,...
70245,Why I think UBER is a BUY,0,1h2kshj,52,2024-11-29 12:32:18,**Personal trading context:**\n\nI do not look...,lzjzffi,Who can make self-driving network and operate ...,1,Uber
70246,Why I think UBER is a BUY,0,1h2kshj,52,2024-11-29 12:32:18,**Personal trading context:**\n\nI do not look...,lzn41d8,"Relax, he is a ""Joseph Carlson"" bot. Username ...",1,Uber
70247,Why I think UBER is a BUY,0,1h2kshj,52,2024-11-29 12:32:18,**Personal trading context:**\n\nI do not look...,lzl059p,"> Uber might, but Tesla also might not allow t...",2,Uber
70248,Why I think UBER is a BUY,0,1h2kshj,52,2024-11-29 12:32:18,**Personal trading context:**\n\nI do not look...,lzjyx6d,I have for 3 I'll let u know in another 2. not...,0,Uber


In [62]:
keywords = ["Uber", "youtube", "meta", "apple", "nvidia", "microsoft", "amazon", "Saudi Arabian Oil Co", "intel", "tesla", "MARA Holdings"]
company_dfs = {company: full_df[full_df['company'] == company] for company in full_df['company'].unique()}
for company_name in keywords:
    company_df = company_dfs.get(company_name)
    
    globals()[company_name] = company_df
    
print(f"DataFrame for tesla created.")

DataFrame for tesla created.


### Cleaning of comments column

In [ ]:
full_df.